<h1><center><font color='black'>Predicción de cotizaciones de bolsa, usando scikit-learn mediante los métodos de regresión de vecinos próximos y SVR</font></center></h1>

### Alumno: Pablo Esau Mejia Medina
### Asingatura: Gestión de la información

In [2]:
import numpy as np
from sklearn.svm import SVR
from sklearn import preprocessing
from pandas_datareader import data
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split 

<h1><center><font color='red'>Lectura y preprocesamiento de datos</font></center></h1>

* Esta seccion se encarga de calcular el promedio de cambio de cada accion, separa los registros con los que se haran pruebas, ademas divide la data en el conjunto de entrenamiento y test en una proporcion de 80%,20% respectivamente.

* Es importante, aclarar que en esta seccion para la prediccion de cada dia se toma en cuenta el valor del dia anterior.

In [3]:
#Dividimos nuestros datos en "conjunto de entrenamiento y de prueba
Netflix = data.DataReader('NFLX', 'yahoo',start='1/1/2003')
#Se define el número de datos a predecir
pronostico = int(1)
y_exacta = Netflix["Close"][-pronostico]
#Calculo el porcentaje del promedio de cambios durante el día
Netflix['HL'] = (Netflix['High'] - Netflix['Low']) / Netflix['Low'] * 100.0
Netflix['Change'] = (Netflix['Close'] - Netflix['Open']) / Netflix['Open'] * 100.0
Netflix = Netflix[['Close', 'HL', 'Change', 'Volume']]
#Se define la columna con las etiquetas o respuestas
data_etiquetada = 'Close'
#Se crea una nueva columna con el pronostico de la data
Netflix['Pronostico'] = Netflix[data_etiquetada].shift(-pronostico)
#Se crea el conjunto de datos con las características
X = np.array(Netflix.drop(['Pronostico'], 1))
X = preprocessing.scale(X)
#Se elimina las filas de los datos a predecir
X = X[:-pronostico]
#Se crea una variable que contiene solamente los datos para predecir
X_prediccion = X[-pronostico:]
#Se crea una variable con las etiquetas o respuestas
y = np.array(Netflix['Pronostico'])
#Se elimina las filas de los datos a predecir
y = y[:-pronostico]
#Se selecciona el 20% de la data entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

<h1><center><font color='red'>Modelo de regresion con K vecinos (KNN-Regresion)</font></center></h1>

In [4]:
knn = KNeighborsRegressor(n_neighbors=5) #Declaramos al método
modelKnn = knn.fit(x_train, y_train) #Ajustamos a al método
confianzaKnn = modelKnn.score(x_test, y_test) # El porcentaje de acertamiento del método
print("Confianza del modelo: ",confianzaKnn*100 )

Confianza del modelo:  99.09328495661286


### La funcion score da un coeficiente que en $R ^ 2$  se define como $(1 - u / v)$, donde:

* 1. $u$ es la suma residual de cuadrados ((y_true - y_pred) ** 2) .sum () y 
* 2. v es la suma total de cuadrados ((y_true - y_true.mean ()) ** 2) .sum (). 

La mejor puntuación posible es 1.0 y puede ser negativa (porque el modelo puede ser arbitrariamente peor). Un modelo constante que siempre predice el valor esperado de y, sin tener en cuenta las características de entrada, obtendría una puntuación $R ^ 2$ de 0.0.

## Prueba

In [5]:
print("Valor Predicho para hoy:",modelKnn.predict(X_prediccion))
print("Valor Real para hoy: ")

Valor Predicho para hoy: [316.66199951]
Valor Real para hoy: 


##### El valor real del dia 10 de enero del 2019 es de 319.96 $. [Click para ver el Precio real](https://www.google.es/search?rlz=1C1JZAP_esHN814HN814&ei=-V0KXMukLcyS1fAP-JGXkA8&q=netflix+precio+acciones&oq=netflix+precio+acciones&gs_l=psy-ab.3..0.1343.2361..2419...0.0..0.87.653.8......0....1..gws-wiz.......0i71j0i131j0i67j0i22i30.fzb9bUZVgIY)

<h1><center><font color='red'>Modelo de regresion con SVR</font></center></h1>

In [6]:
clf = SVR(kernel='rbf', C=1e3, gamma=0.1)
modelSVR = clf.fit(x_train, y_train)
confianzaSVR=modelSVR.score(x_test, y_test)
print("Confianza del modelo: ",confianzaSVR*100)

Confianza del modelo:  99.73732077018471


In [7]:
print("Valor Predicho para hoy:",modelSVR.predict(X_prediccion))
print("Valor Real para hoy: ")

Valor Predicho para hoy: [320.66623914]
Valor Real para hoy: 


##### El valor real del dia 10 de enero del 2019 es de 319.96 $. [Click para ver el Precio real](https://www.google.es/search?rlz=1C1JZAP_esHN814HN814&ei=-V0KXMukLcyS1fAP-JGXkA8&q=netflix+precio+acciones&oq=netflix+precio+acciones&gs_l=psy-ab.3..0.1343.2361..2419...0.0..0.87.653.8......0....1..gws-wiz.......0i71j0i131j0i67j0i22i30.fzb9bUZVgIY)

<h1><center><font color='red'>Considerando dos dias anteriores</font></center></h1>

* En esta seccion se considera 2 dias anteriores para predecir el valor de un dia determinado.

In [8]:
#Dividimos nuestros datos en "conjunto de entrenamiento y de prueba
Netflix = data.DataReader('NFLX', 'yahoo',start='1/1/2003')
#Se define el número de datos a predecir
pronostico = int(1)
y_exacta = Netflix["Close"][-pronostico]
#Calculo el porcentaje del promedio de cambios durante el día
Netflix['HL'] = (Netflix['High'] - Netflix['Low']) / Netflix['Low'] * 100.0
Netflix['Change'] = (Netflix['Close'] - Netflix['Open']) / Netflix['Open'] * 100.0
Netflix = Netflix[['Close', 'HL', 'Change', 'Volume']]
#Se define la columna con las etiquetas o respuestas
data_etiquetada = 'Close'
#Se crea una nueva columna con el pronostico de la data
Netflix['Pronostico'] = Netflix[data_etiquetada].shift(-pronostico)
#Se crea el conjunto de datos con las características
X = np.array(Netflix.drop(['Pronostico'], 1))
X = preprocessing.scale(X)
#Se elimina las filas de los datos a predecir
X = X[:-pronostico]
#################################################################
X_par = X[::2]
X_impar = X[1::2] 
if(len(X_par) > len(X_impar)):
    X_par = np.delete(X_par, len(X_par)-1, 0)
if(len(X_par) < len(X_impar)):
    X_impar = np.delete(X_impar, len(X_impar)-1, 0)
    
X=np.concatenate((X_par, X_impar), axis=1) 
X = X[:-pronostico]
##################################################################
#Se crea una variable que contiene solamente los datos para predecir
X_prediccion = X[-pronostico:]
#Se crea una variable con las etiquetas o respuestas
y = np.array(Netflix['Pronostico'])
y = y[2::2]
#Se elimina las filas de los datos a predecir
y = y[:-pronostico]
#Se selecciona el 20% de la data entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

<h1><center><font color='red'>Modelo de regresion con K vecinos (KNN-Regresion)</font></center></h1>

In [9]:
knn = KNeighborsRegressor(n_neighbors=5) #Declaramos al método
modelKnn = knn.fit(x_train, y_train) #Ajustamos a al método
confianzaKnn = modelKnn.score(x_test, y_test) # El porcentaje de acertamiento del método
print("Confianza del modelo: ",confianzaKnn*100 )

Confianza del modelo:  98.10441943509655


In [10]:
print("Valor Predicho para hoy:",modelKnn.predict(X_prediccion))
print("Valor Real para hoy: ")

Valor Predicho para hoy: [310.65]
Valor Real para hoy: 


##### El valor real del dia 10 de enero del 2019 es de 319.96 $. [Click para ver el Precio real](https://www.google.es/search?rlz=1C1JZAP_esHN814HN814&ei=-V0KXMukLcyS1fAP-JGXkA8&q=netflix+precio+acciones&oq=netflix+precio+acciones&gs_l=psy-ab.3..0.1343.2361..2419...0.0..0.87.653.8......0....1..gws-wiz.......0i71j0i131j0i67j0i22i30.fzb9bUZVgIY)

<h1><center><font color='red'>Modelo de regresion con SVR</font></center></h1>

In [11]:
clf = SVR(kernel='rbf', C=1e3, gamma=0.1)
modelSVR = clf.fit(x_train, y_train)
confianzaSVR=modelSVR.score(x_test, y_test)
print("Confianza del modelo: ",confianzaSVR*100 )

Confianza del modelo:  99.10953503203766


In [12]:
print("Valor Predicho para hoy:",modelSVR.predict(X_prediccion))
print("Valor Real para hoy: ")

Valor Predicho para hoy: [320.17016953]
Valor Real para hoy: 


##### El valor real del dia 10 de enero del 2019 es de 319.96 $. [Click para ver el Precio real](https://www.google.es/search?rlz=1C1JZAP_esHN814HN814&ei=-V0KXMukLcyS1fAP-JGXkA8&q=netflix+precio+acciones&oq=netflix+precio+acciones&gs_l=psy-ab.3..0.1343.2361..2419...0.0..0.87.653.8......0....1..gws-wiz.......0i71j0i131j0i67j0i22i30.fzb9bUZVgIY)

<h1><center><font color='red'>Concluciones</font></center></h1>

* De los resultados anteriores vemos que el modelo SVR da una mejor confianza aproximada de 99.73% frente a 99.09% del knn-Regresión al momento de predecir el precio de las acciones.
* Al intentar mejorar la predicion hecha por el modelo Knn-Regresión considerando dos días anteriores para predecir el preio de la acción actual, da como resultado que el modelo se entrena peor. De manera, que este enfoque no funciono.
* Por último, se recomienda usar SVR por el nivel de confianza que este presenta para realizar predicciones acerca de valores en la bolsa.